In [ ]:
"""
Grade_num may be added or reduced, described by type.
Find the user with the highest increase on his/her grade_num (possibly more than 1 user) 
Show id, name and total grade_num of the user(s)  and order the result by id ascending
"""

In [ ]:
"""SQL table creation and data insertion"""
drop table if exists user;
drop table if exists grade_info;

CREATE TABLE user (
id  int(4) NOT NULL,
name varchar(32) NOT NULL
);

CREATE TABLE grade_info (
user_id  int(4) NOT NULL,
grade_num int(4) NOT NULL,
type varchar(32) NOT NULL
);

INSERT INTO user VALUES
(1,'tm'),
(2,'wwy'),
(3,'zk'),
(4,'qq'),
(5,'lm');

INSERT INTO grade_info VALUES
(1,3,'add'),
(2,3,'add'),
(1,1,'reduce'),
(3,3,'add'),
(4,3,'add'),
(5,3,'add'),
(3,1,'reduce');

In [ ]:
"""SQL solutiuon"""
# 1
SELECT u.id, u.name, a.grade_sum
FROM 
(SELECT user_id, SUM(CASE WHEN type = 'add' THEN grade_num ELSE grade_num*(-1) END) AS grade_sum
FROM grade_info
GROUP BY user_id) AS a, user AS u
WHERE a.grade_sum =
(SELECT MAX(gs)
FROM
(SELECT SUM(CASE WHEN type = 'add' THEN grade_num ELSE grade_num*(-1) END) OVER(PARTITION BY user_id) AS gs
FROM grade_info) AS b)
AND a.user_id = u.id

# 2
SELECT a.user_id, u.name, a.grade_sum
FROM
(SELECT user_id, SUM(IF(type = 'add', grade_num, -1*grade_num)) AS grade_sum
FROM grade_info
GROUP BY user_id) AS a, user AS u
WHERE a.user_id = u.id
AND a.grade_sum =
(SELECT MAX(gs)
FROM
(SELECT SUM(IF(type = 'add', grade_num, -1*grade_num)) OVER(PARTITION BY user_id) AS gs
 FROM grade_info) AS b)
ORDER BY a.user_id

In [1]:
"""pandas dataframe creation"""
import pandas as pd

tu = [(1,'tm'),
(2,'wwy'),
(3,'zk'),
(4,'qq'),
(5,'lm')]

gi = [(1,3,'add'),
(2,3,'add'),
(1,1,'reduce'),
(3,3,'add'),
(4,3,'add'),
(5,3,'add'),
(3,1,'reduce')]

user = pd.DataFrame(tu, columns=['id', 'name'])
grade_info = pd.DataFrame(gi, columns=['user_id', 'grade_num', 'type'])
user.head()

,id,name
0,1,tm
1,2,wwy
2,3,zk
3,4,qq
4,5,lm


In [2]:
grade_info.head()

,user_id,grade_num,type
0,1,3,add
1,2,3,add
2,1,1,reduce
3,3,3,add
4,4,3,add


In [12]:
grade_info['grade_change'] = 0
grade_info['grade_change'] = grade_info['grade_num'] * ((grade_info['type'] == 'add') + -1 * (grade_info['type'] == 'reduce'))
grade_info

,user_id,grade_num,type,grade_change
0,1,3,add,3
1,2,3,add,3
2,1,1,reduce,-1
3,3,3,add,3
4,4,3,add,3
5,5,3,add,3
6,3,1,reduce,-1


In [13]:
df = grade_info.groupby('user_id')['grade_change'].sum().reset_index().rename(columns={'grade_change': 'grade_sum'})
df

,user_id,grade_sum
0,1,2
1,2,3
2,3,2
3,4,3
4,5,3


In [14]:
df = pd.merge(df, user, left_on='user_id', right_on='id', suffixes=['', '_user'])
df

,user_id,grade_sum,id,name
0,1,2,1,tm
1,2,3,2,wwy
2,3,2,3,zk
3,4,3,4,qq
4,5,3,5,lm


In [15]:
df[df['grade_sum'] == df['grade_sum'].max()][['user_id', 'name', 'grade_sum']].sort_values(by='user_id')

,user_id,name,grade_sum
1,2,wwy,3
3,4,qq,3
4,5,lm,3
